In [9]:
from __future__ import print_function
import gensim
import keras
from keras.callbacks import TensorBoard
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, LSTM, GRU, Embedding, Flatten, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

加载词嵌入矩阵、数据

In [2]:
embedding_model = gensim.models.KeyedVectors.load_word2vec_format('../models/word2vec.bin', binary=True)

In [3]:
data = pd.read_csv("../datasets/train_processed.csv")

# 生成embedding字典

In [4]:
word2index = dict(zip(embedding_model.index2word, range(1, len(embedding_model.index2word) + 1)))

In [5]:
def translate_word_to_index(sequence):
    return [word2index[w] if w in word2index else 0 for w in sequence]

In [6]:
embedding_metric = np.array([embedding_model[word] for word, _ in word2index.items()])

In [7]:
X1 = pad_sequences([translate_word_to_index(s) for s in data.question1_words.values], 50)
X2 = pad_sequences([translate_word_to_index(s) for s in data.question2_words.values], 50)
y = data.is_duplicate.values

/Users/heyao/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  


In [10]:
X1_train, X1_val, X2_train, X2_val, y_train, y_val = train_test_split(X1, X2, y, test_size=0.2, random_state=42)
print("X shape:\n", "X1 train:", X1_train.shape, "X1 val:", X1_val.shape, "X2 train:", X2_train.shape, "X2 val:", X2_val.shape)
print("y shape:\n", "y train:", y_train.shape, "y val:", y_val.shape)

X shape:
 X1 train: (323429, 50) X1 val: (80858, 50) X2 train: (323429, 50) X2 val: (80858, 50)
y shape:
 y train: (323429,) y val: (80858,)


# 构建模型

In [11]:
MAX_SEQUENCE_LEN = 50
WORD_DIM = 128
num_words = len(embedding_metric) + 1

## CNN

In [15]:
# cnn
q_a = Input(shape=(MAX_SEQUENCE_LEN,))
q_b = Input(shape=(MAX_SEQUENCE_LEN,))
shared_embedding = Embedding(input_dim=num_words, output_dim=WORD_DIM, 
                             embeddings_initializer=keras.initializers.Constant(embedding_metric), 
                             input_length=MAX_SEQUENCE_LEN, trainable=False)
# shared_lstm = LSTM(32, activation='relu')
embed_a = shared_embedding(q_a)
embed_b = shared_embedding(q_b)
# encoded_a = shared_lstm(embed_a)
# encoded_b = shared_lstm(embed_b)
shared_cnn1 = Conv1D(16, 2, padding='same', activation='relu')
shared_cnn2 = Conv1D(32, 2, padding='same', activation='relu')
shared_cnn3 = Conv1D(64, 2, padding='same', activation='relu')
# shared_cnn4 = Conv1D(128, 2, padding='same', activation='relu')
shared_pool1 = MaxPooling1D(2, strides=1)
shared_pool2 = MaxPooling1D(2, strides=1)
shared_pool3 = MaxPooling1D(2, strides=1)
# shared_pool4 = MaxPooling1D(2, strides=1)
cnn_a1 = shared_pool1(shared_cnn1(embed_a))
cnn_b1 = shared_pool1(shared_cnn1(embed_b))
cnn_a2 = shared_pool2(shared_cnn2(cnn_a1))
cnn_b2 = shared_pool2(shared_cnn2(cnn_b1))
# cnn_a3 = shared_pool3(shared_cnn3(cnn_a2))
# cnn_b3 = shared_pool3(shared_cnn3(cnn_b2))
encoded_a = Flatten()(shared_pool3(shared_cnn3(cnn_a2)))
encoded_b = Flatten()(shared_pool3(shared_cnn3(cnn_b2)))

merged_vec = keras.layers.concatenate([encoded_a, encoded_b])
hidden1 = Dense(64, activation='relu')(merged_vec)
hidden1_drop = Dropout(0.5)(hidden1)
output = Dense(1, activation='sigmoid')(hidden1_drop)
model = Model(inputs=[q_a, q_b], outputs=output)

## fully-connected

In [12]:
q_a = Input(shape=(MAX_SEQUENCE_LEN,))
q_b = Input(shape=(MAX_SEQUENCE_LEN,))
shared_embedding = Embedding(input_dim=num_words, output_dim=WORD_DIM, 
                             embeddings_initializer=keras.initializers.Constant(embedding_metric), 
                             input_length=MAX_SEQUENCE_LEN, trainable=False)

embed_a = shared_embedding(q_a)
embed_b = shared_embedding(q_b)
shared_fc1 = Dense(64, activation='relu')
shared_fc2 = Dense(32, activation='relu')
shared_fc3 = Dense(8, activation='relu')
fc_a1 = shared_fc1(embed_a)
fc_b1 = shared_fc1(embed_b)
fc_a2 = shared_fc2(fc_a1)
fc_b2 = shared_fc2(fc_a1)
fc_a3 = shared_fc3(fc_a2)
fc_b3 = shared_fc3(fc_a2)
merged_vec = Flatten()(keras.layers.concatenate([fc_a3, fc_b3]))
hidden1 = Dense(8, activation='relu')(merged_vec)
output = Dense(1, activation='sigmoid')(hidden1)
model = Model(inputs=[q_a, q_b], outputs=output)

In [16]:
model.compile('adam', loss='binary_crossentropy', metrics=['acc'])

In [17]:
model.fit([X1_train, X2_train], y_train, batch_size=256, epochs=50, 
          validation_data=([X1_val, X2_val], y_val))  # , callbacks=[TensorBoard('../logs')]

Train on 323429 samples, validate on 80858 samples
Epoch 1/50
323429/323429 [==============================] - 149s 461us/step - loss: 0.6336 - acc: 0.6513 - val_loss: 0.6030 - val_acc: 0.6814
Epoch 2/50
323429/323429 [==============================] - 188s 581us/step - loss: 0.5972 - acc: 0.6877 - val_loss: 0.5767 - val_acc: 0.7012
Epoch 3/50
323429/323429 [==============================] - 162s 501us/step - loss: 0.5731 - acc: 0.7066 - val_loss: 0.5551 - val_acc: 0.7138
Epoch 4/50
323429/323429 [==============================] - 163s 504us/step - loss: 0.5561 - acc: 0.7181 - val_loss: 0.5495 - val_acc: 0.7193
Epoch 5/50
323429/323429 [==============================] - 164s 508us/step - loss: 0.5433 - acc: 0.7268 - val_loss: 0.5385 - val_acc: 0.7244
Epoch 6/50
323429/323429 [==============================] - 160s 495us/step - loss: 0.5323 - acc: 0.7336 - val_loss: 0.5254 - val_acc: 0.7401
Epoch 7/50
323429/323429 [==============================] - 169s 524us/step - loss: 0.5226 - acc:

In [18]:
model.save('../models/baseline_cnn3_fc64_drop.hdf5')